# 🏙️ Análisis Exploratorio de Airbnb - Barcelona

Este proyecto realiza un **EDA (Exploratory Data Analysis)** del mercado de alquileres de Airbnb en Barcelona.

**Objetivos principales:**
- Analizar la distribución de precios y disponibilidad.
- Identificar los barrios más caros y con más actividad.
- Explorar patrones geográficos y temporales.
- Preparar datos para futuros modelos predictivos.

**Estructura del notebook:**
1. Carga y limpieza de datos (`src/utils.py`)
2. Análisis descriptivo
3. Exploración geográfica
4. Tendencias temporales
5. Conclusiones

In [ ]:
# Configuración general
import sys, os
sys.path.append(os.path.abspath('..'))  # Permite importar desde /src

# Utils personalizadas
from src.utils import (
    load_listings,
    clean_df,
    clean_neighbourhoods,
    top_neighbourhoods
)

# Librerías estándar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

# Configuración visual
plt.style.use('seaborn-v0_8')
sns.set_palette('muted')

print("✅ Librerías cargadas correctamente.")

In [ ]:
# Cargar el dataset original
df = load_listings('../data/listings.csv')

print(f"✅ Dataset cargado correctamente con {df.shape[0]:,} filas y {df.shape[1]} columnas.")
df.head(3)

In [ ]:
df = clean_df(df)

print("✅ Limpieza básica completada.")
print(f"Columnas después de limpieza: {df.shape[1]}")
df.info()

In [ ]:
# Identificar columna de barrio
if 'neighbourhood_cleansed' in df.columns:
    neigh_col = 'neighbourhood_cleansed'
elif 'neighbourhood' in df.columns:
    neigh_col = 'neighbourhood'
else:
    neigh_col = None
    print("⚠️ No se encontró una columna de barrio.")

# Limpiar nombres de barrios
if neigh_col:
    df = clean_neighbourhoods(df, col=neigh_col)
    print(f"✅ Barrios limpios en columna '{neigh_col}'")
    display(df[neigh_col].value_counts().head(10))

In [ ]:
# Filtrar precios extremos
price_cap = 2000
df_filtered = df[df['price'] <= price_cap]

plt.figure(figsize=(10,6))
sns.histplot(df_filtered['price'], bins=60, kde=True, color='teal')

# Calcular estadísticas
mean_price = df_filtered['price'].mean()
median_price = df_filtered['price'].median()

# Líneas de media y mediana
plt.axvline(mean_price, color='red', linestyle='--', label=f"Media: {mean_price:.0f}€")
plt.axvline(median_price, color='blue', linestyle='--', label=f"Mediana: {median_price:.0f}€")

# Título y etiquetas
plt.title("Distribución de precios por noche (hasta 2000 €)", fontsize=14, weight='bold')
plt.xlabel("Precio (€)", fontsize=12)
plt.ylabel("Número de alojamientos", fontsize=12)
plt.legend()
plt.show()

print(f"Media: {mean_price:.2f} € | Mediana: {median_price:.2f} € | Máximo mostrado: {price_cap} €")

In [ ]:
if neigh_col:
    # Obtener los barrios con más de 10 listados
    top_neigh = top_neighbourhoods(df, neigh_col=neigh_col, min_listings=10, top_n=20)
    
    # Ordenar correctamente para que el gráfico horizontal sea legible
    top_neigh = top_neigh.sort_values('mean_price', ascending=True)

    plt.figure(figsize=(10, 8))
    bars = plt.barh(top_neigh[neigh_col], top_neigh['mean_price'], color='steelblue')

    # Etiquetas de valor al final de cada barra
    for bar in bars:
        plt.text(
            bar.get_width() + 15,               # posición X
            bar.get_y() + bar.get_height()/2,   # posición Y
            f"{bar.get_width():.0f} €",         # texto
            va='center',
            fontsize=9,
            color='black'
        )

    plt.title("Top 20 barrios por precio medio (Airbnb Barcelona)", fontsize=14, weight='bold', pad=15)
    plt.xlabel("Precio medio (€)")
    plt.ylabel("Barrio")
    plt.grid(axis='x', linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()

In [ ]:
# Variables a analizar
corr_cols = ['price', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'availability_365']
corr_cols = [c for c in corr_cols if c in df.columns]

# Calcular matriz de correlación
corr = df[corr_cols].corr()

# Traducir nombres de variables
col_names = {
    'price': 'Precio (€)',
    'minimum_nights': 'Noches mínimas',
    'number_of_reviews': 'Nº de reseñas',
    'reviews_per_month': 'Reseñas/mes',
    'availability_365': 'Disponibilidad (días/año)'
}

corr = corr.rename(index=col_names, columns=col_names)

# Crear gráfico
plt.figure(figsize=(8,6))
sns.heatmap(
    corr,
    annot=True,
    fmt=".2f",
    cmap="YlGnBu",         # Colores más agradables y profesionales
    linewidths=0.5,
    cbar_kws={'label': 'Correlación'}
)

plt.title("Correlación entre variables numéricas", fontsize=14, weight='bold', pad=15)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
if 'last_review' in df.columns:
    # Asegurar que las fechas sean válidas
    df['last_review'] = pd.to_datetime(df['last_review'], errors='coerce')
    
    # Contar reseñas por año
    timeline = (
        df.dropna(subset=['last_review'])
          .set_index('last_review')
          .resample('Y')
          .size()
          .rename('n_reviews')
    )

    # Reiniciar índice para plot
    timeline = timeline.reset_index()
    timeline['Año'] = timeline['last_review'].dt.year
    timeline = timeline[timeline['Año'] >= 2010]  # Mostrar desde 2010 para acotar

    # Crear figura
    plt.figure(figsize=(10,6))
    sns.lineplot(
        data=timeline,
        x='Año',
        y='n_reviews',
        marker='o',
        color='teal'
    )

    # Anotaciones con valores
    for x, y in zip(timeline['Año'], timeline['n_reviews']):
        plt.text(x, y + timeline['n_reviews'].max() * 0.02, f"{y:,}", ha='center', fontsize=9)

    # Título y ejes
    plt.title("Evolución anual del número de reseñas (Airbnb Barcelona)", fontsize=14, weight='bold', pad=15)
    plt.xlabel("Año")
    plt.ylabel("Número de reseñas")
    plt.grid(alpha=0.4, linestyle='--')
    plt.tight_layout()
    plt.show()

    print(f"Periodo mostrado: {timeline['Año'].min()} – {timeline['Año'].max()}")

In [ ]:
if {'latitude', 'longitude'}.issubset(df.columns):
    import folium
    from folium.plugins import MarkerCluster
    from IPython.display import display

    # Crear muestra y mapa
    map_df = df.dropna(subset=['latitude', 'longitude']).sample(1000, random_state=42)
    center = [map_df['latitude'].mean(), map_df['longitude'].mean()]

    m = folium.Map(location=center, zoom_start=12, tiles="CartoDB positron")
    cluster = MarkerCluster().add_to(m)

    for _, row in map_df.iterrows():
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=3,
            popup=f"{row.get('name','')[:50]} - {row.get('price','N/A')}€",
            fill=True,
            color='teal',
            fill_opacity=0.7
        ).add_to(cluster)

    # Guardar mapa en HTML
    import os
    os.makedirs('../reports/figures', exist_ok=True)
    m.save('../reports/figures/map_sample.html')

    # Mostrar el mapa directamente en el notebook
    display(m)

In [ ]:
df.to_csv('../data/listings_clean.csv', index=False)
print("✅ Dataset limpio guardado en '../data/listings_clean.csv'")

## 🧭 Conclusiones principales

- Los precios más altos se concentran en **Sarrià-Sant Gervasi** y **Eixample**.  
- Las zonas con más actividad de reseñas son **Ciutat Vella** y **Gràcia**.  
- La mayoría de los alojamientos cuestan entre **50€ y 150€ por noche**.  
- Propiedades más caras tienden a tener **menos reseñas**.  
- Fuerte crecimiento de reseñas entre 2016–2019, caída en 2020 (COVID).  
- El mercado muestra clara concentración en el centro histórico y zonas turísticas.